<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hu_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gn0__iru
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gn0__iru
  Resolved https://github.com/huggingface/transformers to commit 25b0f2033ba23e354ef2f665764248fcbb3f49ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [37]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [38]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [39]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hu", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [40]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("tiny")

In [41]:
model = whisper.load_model("tiny")

In [42]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [43]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hu")
    return result["text"]

In [44]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [45]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [46]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.2666666666666666, Precision: 0.19047619047619047, Recall: 0.26666666666666666, F1: 0.2222222222222222
Processed 2 files, WER: 0.5909090909090909, Precision: 0.5, Recall: 0.6, F1: 0.5454545454545454
Processed 3 files, WER: 0.8157894736842105, Precision: 0.2777777777777778, Recall: 0.30303030303030304, F1: 0.28985507246376807
Processed 4 files, WER: 0.782608695652174, Precision: 0.2916666666666667, Recall: 0.3181818181818182, F1: 0.30434782608695654
Processed 5 files, WER: 0.7647058823529411, Precision: 0.3125, Recall: 0.29411764705882354, F1: 0.30303030303030304
Processed 6 files, WER: 1.0, Precision: 0, Recall: 0.0, F1: 0
Processed 7 files, WER: 0.9375, Precision: 0.23529411764705882, Recall: 0.25, F1: 0.24242424242424243
Processed 8 files, WER: 0.8235294117647058, Precision: 0.3125, Recall: 0.37037037037037035, F1: 0.3389830508474576
Processed 9 files, WER: 2.358490566037736, Precision: 0.0975609756097561, Recall: 0.1, F1: 0.09876543209876544
Processed 10 fil